In [7]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
import time

load_dotenv()
client = OpenAI()

In [8]:
system_prompt = """
Bạn là trợ lý tạo dữ liệu QA. Nhiệm vụ: Tạo 100 cặp Q&A từ thông tin người dùng cung cấp với các yêu cầu:
1. Câu hỏi phải đa dạng: hỏi sự kiện, định nghĩa, so sánh, nguyên nhân-kết quả
2. Câu trả lời ngắn gọn (1-2 câu), chính xác với nội dung gốc
3. Định dạng nghiêm ngặt: 
   "Q1: [Nội dung]"
   "A1: [Trả lời]"
   "Q2: ..."
   "A2: ..."
4. Chỉ sử dụng thông tin từ input
5. Tạo đủ 100 cặp Q&A
6. Các cặp phải độc lập với nhau
"""

content_with_weight = [
    "Quyền lợi của sinh viên 1. Được hưởng đầy đủ chế độ, chính sách hiện hành của Đảng, Nhà nước cũng như các quy chế, quy định của Đại học Quốc gia Hà Nội và của đơn vị đào tạo. 2. Được phép thôi học vì lý do chủ quan của cá nhân, trong trường hợp này, sinh viên phải hoàn trả cho đơn vị đào tạo toàn bộ kinh phí đào tạo từ ngân sách nhà nước trong thời gian theo học. 3. Sinh viên được xin nghỉ học tạm thời và bảo lưu kết quả đã học trong các trường hợp sau đây: a) Được động viên vào lực lượng vũ trang; b) Được cơ quan có thẩm quyền điều động, đại diện quốc gia tham dự các kỳ thi, giải đấu quốc tế; c) Bị ốm, thai sản hoặc tai nạn phải điều trị thời gian dài có chứng nhận của cơ sở khám bệnh, chữa bệnh có thẩm quyền theo quy định của Bộ Y tế; d) Vì lý do cá nhân khác nhưng phải học tối thiểu 01 học kỳ và không thuộc các trường hợp bị xem xét buộc thôi học hoặc xem xét kỷ luật và phải đạt điểm trung bình chung tích lũy tổi thiểu 2,0. Thời gian nghỉ học tạm thời vì nhu cầu cá nhân được tính vào thời gian tối đa được phép học. 4. Ngoài những quyền lợi chung đối với sinh viên chương trình đào tạo chuẩn, sinh viên thuộc chương trình đào tạo tài năng, chất lượng cao còn được hưởng các quyền lợi sau: a) Được các nhà khoa học đầu ngành, các giáo sư, giảng viên có trình độ, kinh nghiệm và có uy tín trong nước, quốc tế trực tiếp giảng dạy, hướng dẫn nghiên cứu khoa học; b) Được ưu tiên cung cấp hoặc sử dụng các tài liệu học tập; được ưu tiên sử dụng các phương tiện, trang thiết bị kỹ thuật, thư viện và hệ thống internet phục vụ cho học tập và nghiên cứu khoa học; c) Được ưu tiên xét cấp học bổng khuyến khích phát triển và học bổng của các tổ chức, cá nhân trong, ngoài nước. Những sinh viên ở xa được ưu tiên bố trí chỗ ở trong ký túc xá; d) Được ưu tiên xét chọn đi học ở nước ngoài hoặc theo các chương trình hợp tác quốc tế của Đại học Quốc gia Hà Nội, của đơn vị đào tạo.",
    "Chương trình đào tạo Kỹ thuật năng lượng Chương trình đào tạo nguồn nhân lực kỹ sư ngành Kỹ thuật năng lượng (KTNL) nắm vững các kiến thức cơ bản và chuyên sâu về năng lượng, có khả năng thực hành tốt, ứng dụng các hiểu biết và kỹ năng thực hành để nghiên cứu, triển khai thực tế. Kỹ sư tốt nghiệp ngành KTNL có khả năng làm việc tại các tập đoàn trong nước, công ty nước ngoài hoạt động trong lĩnh vực năng lượng mới và truyền thóng, các trường đạo học và viện nghiên cứu có liên quan. Về kiến thức: - Trang bị kiến thức có hệ thống và hiện đại, phù hợp với các chương trình đào tạo tiến tiến trên thế giới: Kiến thức tổng hợp về khoa học tự nhiên, khoa học xã hội và nhân văn, ngoại ngữ; kiến thức cơ bản khác như toán – tin, vật lý đại cương, cơ kỹ thuật; kiến thức nền tảng về năng lượng; kiến thức chuyên sâu theo định hướng năng lượng mặt trời… Về năng lực: - Kỹ sư tốt nghiệp ngành KTNL có năng lực áp dụng các kiến thức cơ bản và chuyên sâu để nghiên cứu, phân tích, thiết kế, triển khai, quản trị các công nghệ và hệ thống năng lượng; năng lực thiết kế, chế tạo, thử nghiệm vật liệu, thiết bị dùng trong kỹ thuật năng lượng, hệ thống năng lượng…; năng lực làm việc ở các vị trí kỹ thuật triển khai và vận hành… Về thái độ: - Đảm bảo sinh viên tốt nghiệp có phẩm chất chính trị tốt; ý thức tổ chức kỷ luật; tinh thần làm việc theo nhóm và khả năng giao tiếp tốt.",
    "Chương trình đào tạo Công nghệ hàng không vũ trụ Sinh viên ngành Công nghệ Hàng không Vũ trụ sau khi tốt nghiệp có khả năng tham mưu tư vấn và có năng lực thực hiện nhiệm vụ với tư cách như một kỹ sư trong lĩnh vực Hàng không Vũ trụ, đáp ứng các yêu cầu cao về nghiên cứu và ứng dụng của xã hội. Các kỹ sư này có khả năng làm việc độc lập, tự học, tự nghiên cứu và nâng cao trình độ. Thêm vào đó, họ có khả năng thích nghi cao với sự phát triển nhanh chóng của ngành Công nghệ Hàng không Vũ trụ. Các vị trí công tác có thể đảm nhận: - Kỹ sư thiết kế, chế tạo các thiết bị bay, vệ tinh; - Kỹ sư vận hành, bảo dưỡng thiết bị bay; - Kỹ sư thiết kế, lập trình các hệ thống điều khiển, hệ thống nhúng; - Kỹ sư xây dựng hệ thống giám sát không gian, xử lý ảnh viễn thám.",
    "Giải mã ngành học: CN10 - Công nghệ nông nghiệp Tổng quan: - Thời gian đào tạo: 4,5 năm - Hệ đào tạo: Kỹ sư Giới thiệu: - Ngành Công nghệ nông nghiệp đào tạo kỹ sư công nghệ có kiến thức liên ngành về công nghệ thông tin, điện tử viễn thông, điều khiển tự động, công nghệ sinh học và công nghệ nano để ứng dụng các công nghệ tiên tiến này vào nông nghiệp; góp phần tạo ra các sản phẩm nông nghiệp có năng suất, giá trị cao, an toàn và có sức cạnh tranh trên thị trường trong nước và quốc tế. - Sinh viên được đào tạo theo nguyên tắc “học bằng làm”, được thực hành tại các phòng thí nghiệm công nghệ tiên tiến và Trung tâm thực nghiệm Nông nghiệp thông minh Các định hướng đào tạo: - Nông nghiệp kỹ thuật số - Công nghệ sinh học nông nghiệp Vị trí việc làm sau khi ra trường: - Kỹ sư công nghệ nông nghiệp kỹ thuật số tại các doanh nghiệp sản xuất, kinh doanh và sử dụng các thiết bị nông nghiệp công nghệ cao: chuyên thiết kế, chế tạo, lập trình, lắp đặt, vận hành hệ thống và thiết bị nông nghiệp công nghệ cao dựa trên nền tảng kỹ thuật điều khiển, công nghệ thông tin và truyền thông. - Kỹ sư công nghệ sinh học nông nghiệp: sử dụng công nghệ cao để nhân giống, trồng cây trong vườn ươm, chuyển giao công nghệ, tạo các chế phẩm sinh học nông nghiệp công nghệ mới, bảo quản chế biến nông sản, kiểm định chất lượng nông sản. - Cán bộ quản lý dự án nông nghiệp công nghệ cao, cán bộ quản lý và tư vấn chính sách về nông nghiệp công nghệ cao ở các sở/ bộ Khoa học Công nghệ và sở/bộ Nông nghiệp và phát triển nông thôn. - Nghiên cứu viên và giảng viên về lĩnh vực công nghệ và công nghệ nông nghiệp trong các cơ sở giáo dục và cơ sở nghiên cứu về nông nghiệp và công nghệ. - Khởi nghiệp đổi mới sáng tạo trong lĩnh vực công nghệ và công nghệ nông nghiệp",
    "Giới thiệu chung về khoa công nghệ xây dựng - giao thông Mục tiêu tổng quan của chương trình đào tạo Kỹ sư ngành Công nghệ Kỹ thuật Xây dựng là đào tạo cho sinh viên kiến thức về kỹ thuật, công nghệ quy hoạch, thiết kế, thi công, vận hành và quản lý các công trình xây dựng, cơ sở hạ tầng. Trên cơ sở đó, sinh viên có thể nghiên cứu, ứng dụng kiến thức chuyên ngành, liên ngành và thực tế xã hội để giải quyết các vấn đề liên quan đến các công trình xây dựng và kỹ hệ thống cơ sở hạ tầng đô thị, vùng hiện nay đang gặp phải. Sinh viên tốt nghiệp có năng lực tư duy độc lập, sáng tạo, có khả năng làm việc tại các tổng công ty, doanh nghiệp xây dựng trong và ngoài nước; làm công tác quy hoạch, quản lý, thiết kế, thi công, giám sát, quản lý dự án xây dựng trong các bộ ngành và công ty, doanh nghiệp; nghiên cứu khoa học và giảng dạy cho các viện nghiên cứu, trường đại học và làm tiếp thạc sỹ, tiến sĩ tại các trường Đại học trong và ngoài nước.",
    "Quyền lợi của sinh viên 1. Được hưởng đầy đủ chế độ, chính sách hiện hành của Đảng, Nhà nước cũng như các quy chế, quy định của Đại học Quốc gia Hà Nội và của đơn vị đào tạo. 2. Được phép thôi học vì lý do chủ quan của cá nhân, trong trường hợp này, sinh viên phải hoàn trả cho đơn vị đào tạo toàn bộ kinh phí đào tạo từ ngân sách nhà nước trong thời gian theo học. 3. Sinh viên được xin nghỉ học tạm thời và bảo lưu kết quả đã học trong các trường hợp sau đây: a) Được động viên vào lực lượng vũ trang; b) Được cơ quan có thẩm quyền điều động, đại diện quốc gia tham dự các kỳ thi, giải đấu quốc tế; c) Bị ốm, thai sản hoặc tai nạn phải điều trị thời gian dài có chứng nhận của cơ sở khám bệnh, chữa bệnh có thẩm quyền theo quy định của Bộ Y tế; d) Vì lý do cá nhân khác nhưng phải học tối thiểu 01 học kỳ và không thuộc các trường hợp bị xem xét buộc thôi học hoặc xem xét kỷ luật và phải đạt điểm trung bình chung tích lũy tổi thiểu 2,0. Thời gian nghỉ học tạm thời vì nhu cầu cá nhân được tính vào thời gian tối đa được phép học. 4. Ngoài những quyền lợi chung đối với sinh viên chương trình đào tạo chuẩn, sinh viên thuộc chương trình đào tạo tài năng, chất lượng cao còn được hưởng các quyền lợi sau: a) Được các nhà khoa học đầu ngành, các giáo sư, giảng viên có trình độ, kinh nghiệm và có uy tín trong nước, quốc tế trực tiếp giảng dạy, hướng dẫn nghiên cứu khoa học; b) Được ưu tiên cung cấp hoặc sử dụng các tài liệu học tập; được ưu tiên sử dụng các phương tiện, trang thiết bị kỹ thuật, thư viện và hệ thống internet phục vụ cho học tập và nghiên cứu khoa học; c) Được ưu tiên xét cấp học bổng khuyến khích phát triển và học bổng của các tổ chức, cá nhân trong, ngoài nước. Những sinh viên ở xa được ưu tiên bố trí chỗ ở trong ký túc xá; d) Được ưu tiên xét chọn đi học ở nước ngoài hoặc theo các chương trình hợp tác quốc tế của Đại học Quốc gia Hà Nội, của đơn vị đào tạo.",
    "Chương trình đào tạo Kỹ thuật năng lượng Chương trình đào tạo nguồn nhân lực kỹ sư ngành Kỹ thuật năng lượng (KTNL) nắm vững các kiến thức cơ bản và chuyên sâu về năng lượng, có khả năng thực hành tốt, ứng dụng các hiểu biết và kỹ năng thực hành để nghiên cứu, triển khai thực tế. Kỹ sư tốt nghiệp ngành KTNL có khả năng làm việc tại các tập đoàn trong nước, công ty nước ngoài hoạt động trong lĩnh vực năng lượng mới và truyền thóng, các trường đạo học và viện nghiên cứu có liên quan. Về kiến thức: - Trang bị kiến thức có hệ thống và hiện đại, phù hợp với các chương trình đào tạo tiến tiến trên thế giới: Kiến thức tổng hợp về khoa học tự nhiên, khoa học xã hội và nhân văn, ngoại ngữ; kiến thức cơ bản khác như toán – tin, vật lý đại cương, cơ kỹ thuật; kiến thức nền tảng về năng lượng; kiến thức chuyên sâu theo định hướng năng lượng mặt trời… Về năng lực: - Kỹ sư tốt nghiệp ngành KTNL có năng lực áp dụng các kiến thức cơ bản và chuyên sâu để nghiên cứu, phân tích, thiết kế, triển khai, quản trị các công nghệ và hệ thống năng lượng; năng lực thiết kế, chế tạo, thử nghiệm vật liệu, thiết bị dùng trong kỹ thuật năng lượng, hệ thống năng lượng…; năng lực làm việc ở các vị trí kỹ thuật triển khai và vận hành… Về thái độ: - Đảm bảo sinh viên tốt nghiệp có phẩm chất chính trị tốt; ý thức tổ chức kỷ luật; tinh thần làm việc theo nhóm và khả năng giao tiếp tốt.",
    "Chương trình đào tạo Công nghệ hàng không vũ trụ Sinh viên ngành Công nghệ Hàng không Vũ trụ sau khi tốt nghiệp có khả năng tham mưu tư vấn và có năng lực thực hiện nhiệm vụ với tư cách như một kỹ sư trong lĩnh vực Hàng không Vũ trụ, đáp ứng các yêu cầu cao về nghiên cứu và ứng dụng của xã hội. Các kỹ sư này có khả năng làm việc độc lập, tự học, tự nghiên cứu và nâng cao trình độ. Thêm vào đó, họ có khả năng thích nghi cao với sự phát triển nhanh chóng của ngành Công nghệ Hàng không Vũ trụ. Các vị trí công tác có thể đảm nhận: - Kỹ sư thiết kế, chế tạo các thiết bị bay, vệ tinh; - Kỹ sư vận hành, bảo dưỡng thiết bị bay; - Kỹ sư thiết kế, lập trình các hệ thống điều khiển, hệ thống nhúng; - Kỹ sư xây dựng hệ thống giám sát không gian, xử lý ảnh viễn thám.",
    "Giải mã ngành học: CN10 - Công nghệ nông nghiệp Tổng quan: - Thời gian đào tạo: 4,5 năm - Hệ đào tạo: Kỹ sư Giới thiệu: - Ngành Công nghệ nông nghiệp đào tạo kỹ sư công nghệ có kiến thức liên ngành về công nghệ thông tin, điện tử viễn thông, điều khiển tự động, công nghệ sinh học và công nghệ nano để ứng dụng các công nghệ tiên tiến này vào nông nghiệp; góp phần tạo ra các sản phẩm nông nghiệp có năng suất, giá trị cao, an toàn và có sức cạnh tranh trên thị trường trong nước và quốc tế. - Sinh viên được đào tạo theo nguyên tắc “học bằng làm”, được thực hành tại các phòng thí nghiệm công nghệ tiên tiến và Trung tâm thực nghiệm Nông nghiệp thông minh Các định hướng đào tạo: - Nông nghiệp kỹ thuật số - Công nghệ sinh học nông nghiệp Vị trí việc làm sau khi ra trường: - Kỹ sư công nghệ nông nghiệp kỹ thuật số tại các doanh nghiệp sản xuất, kinh doanh và sử dụng các thiết bị nông nghiệp công nghệ cao: chuyên thiết kế, chế tạo, lập trình, lắp đặt, vận hành hệ thống và thiết bị nông nghiệp công nghệ cao dựa trên nền tảng kỹ thuật điều khiển, công nghệ thông tin và truyền thông. - Kỹ sư công nghệ sinh học nông nghiệp: sử dụng công nghệ cao để nhân giống, trồng cây trong vườn ươm, chuyển giao công nghệ, tạo các chế phẩm sinh học nông nghiệp công nghệ mới, bảo quản chế biến nông sản, kiểm định chất lượng nông sản. - Cán bộ quản lý dự án nông nghiệp công nghệ cao, cán bộ quản lý và tư vấn chính sách về nông nghiệp công nghệ cao ở các sở/ bộ Khoa học Công nghệ và sở/bộ Nông nghiệp và phát triển nông thôn. - Nghiên cứu viên và giảng viên về lĩnh vực công nghệ và công nghệ nông nghiệp trong các cơ sở giáo dục và cơ sở nghiên cứu về nông nghiệp và công nghệ. - Khởi nghiệp đổi mới sáng tạo trong lĩnh vực công nghệ và công nghệ nông nghiệp",
    "Giới thiệu chung về khoa công nghệ xây dựng - giao thông Mục tiêu tổng quan của chương trình đào tạo Kỹ sư ngành Công nghệ Kỹ thuật Xây dựng là đào tạo cho sinh viên kiến thức về kỹ thuật, công nghệ quy hoạch, thiết kế, thi công, vận hành và quản lý các công trình xây dựng, cơ sở hạ tầng. Trên cơ sở đó, sinh viên có thể nghiên cứu, ứng dụng kiến thức chuyên ngành, liên ngành và thực tế xã hội để giải quyết các vấn đề liên quan đến các công trình xây dựng và kỹ hệ thống cơ sở hạ tầng đô thị, vùng hiện nay đang gặp phải. Sinh viên tốt nghiệp có năng lực tư duy độc lập, sáng tạo, có khả năng làm việc tại các tổng công ty, doanh nghiệp xây dựng trong và ngoài nước; làm công tác quy hoạch, quản lý, thiết kế, thi công, giám sát, quản lý dự án xây dựng trong các bộ ngành và công ty, doanh nghiệp; nghiên cứu khoa học và giảng dạy cho các viện nghiên cứu, trường đại học và làm tiếp thạc sỹ, tiến sĩ tại các trường Đại học trong và ngoài nước."
]

jsonl_file = "input.jsonl"
with open(jsonl_file, "w", encoding="utf-8") as f:
    for i, content in enumerate(content_with_weight, start=1):
        line = {
            "custom_id": f"request-{i}", 
            "method": "POST", 
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4.1-nano",
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Đây là thông tin đầu vào {i}:\n\n{content}"}
                ],
                "temperature": 0.6
            }
        }
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

print(f"✅ Created JSONL file: {jsonl_file}")

✅ Created JSONL file: input.jsonl


In [9]:
batch_input_file = client.files.create(
    file=open("input.jsonl", "rb"),
    purpose="batch"
)

batch_input_file

FileObject(id='file-2FSFG2XD7FxCh42rh2Fow3', bytes=26076, created_at=1751898806, filename='input.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)

In [10]:
batch_input_file_id = batch_input_file.id
batch_created =client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "Generate QA pairs"
    }
)

In [11]:
print("⏳ Waiting for batch to complete...")
while True:
    batch = client.batches.retrieve(batch_created.id)
    status = batch.status
    print(f"🔄 Status: {status}")
    
    if status in ["completed", "failed", "expired", "cancelled"]:
        break

    time.sleep(10)

⏳ Waiting for batch to complete...
🔄 Status: validating
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: finalizing
🔄 Status: completed


In [12]:
if status == "completed":
    output_file_id = batch.output_file_id

    file_response = client.files.content(output_file_id)
    json_list = [json.loads(line) for line in file_response.text.splitlines()]

    with open("output.json", 'w', encoding='utf-8') as f:
        json.dump(json_list, f, ensure_ascii=False, indent=2)

    print("📁 Output saved to output.json")

elif status == "failed":
    print("❌ Batch failed.")
    if batch.error_file_id:
        print("📎 Download error file:", batch.error_file_id)

else:
    print(f"⚠️ Batch ended with status: {status}")

📁 Output saved to output.json
